In [ ]:
import numpy as np
import ast
import csv
import os

def load_simulation_args(file_path):
    """
    シミュレーション引数（alphaValues, lambdaValues, iterations, maxTime）を args.csv から読み込む。
    
    ファイルフォーマット:
      alphaValues,lambdaValues,iterations,maxTime
      "[0.0, 0.01, ...]","[0.0, 0.01, ...]",50,100

    Returns:
        alpha_values (np.ndarray): α の値の配列
        lambda_values (np.ndarray): λ の値の配列
        iterations (int): 1バッチあたりの反復回数（ネットワーク反復×シミュレーション反復）
        max_time (int): シミュレーションの最大時刻（時刻 0～max_time の max_time+1 個のデータ）
    """
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        row = next(reader)
        alpha_str, lambda_str, iterations_str, max_time_str = row
        alpha_values = np.array(ast.literal_eval(alpha_str))
        lambda_values = np.array(ast.literal_eval(lambda_str))
        iterations = int(iterations_str)
        max_time = int(max_time_str)
    return alpha_values, lambda_values, iterations, max_time

def load_simulation_result(file_pattern, batch_number, num_alpha, num_lambda, iterations, max_time, results_dir=""):
    """
    指定したバッチ番号のシミュレーション結果 CSV を読み込み、4次元配列に変形する。
    
    ファイル名は file_pattern から ".csv" を除き、batch_number を付加したものとする。
    例: "s_all_results.csv" → "s_all_results1.csv", "s_all_results2.csv", ...
    
    Args:
        file_pattern (str): 出力ファイルの基本名（例："s_all_results.csv"）
        batch_number (int): バッチ番号（1-indexed）
        num_alpha (int): α の個数
        num_lambda (int): λ の個数
        iterations (int): 1バッチあたりの反復回数
        max_time (int): シミュレーションの最大時刻（記録は 0～max_time、サイズは max_time+1）
        results_dir (str): CSV ファイルが存在するディレクトリ（省略時はカレントディレクトリ）
    
    Returns:
        np.ndarray: 形状 (num_alpha, num_lambda, iterations, max_time+1) のデータ配列
    """
    filename = file_pattern.replace(".csv", "") + f"{batch_number}.csv"
    file_path = os.path.join(results_dir, filename)

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} not found.")

    # CSV は 1 列 ("value") のみ。ヘッダーをスキップして読み込み
    data = np.loadtxt(file_path, delimiter=',', skiprows=1, dtype=int)
    try:
        data = data.reshape(num_alpha, num_lambda, iterations, max_time+1)
    except ValueError as e:
        raise ValueError(f"Data in {file_path} cannot be reshaped to "
                         f"({num_alpha}, {num_lambda}, {iterations}, {max_time+1}): {e}")
    return data

def load_all_batches(file_pattern, num_batches, num_alpha, num_lambda, iterations, max_time, results_dir=""):
    """
    複数バッチ分のシミュレーション結果を読み込み、反復軸方向に連結する。
    
    Args:
        file_pattern (str): 出力ファイルの基本名（例："s_all_results.csv"）
        num_batches (int): バッチ数
        num_alpha (int): α の個数
        num_lambda (int): λ の個数
        iterations (int): 1バッチあたりの反復回数
        max_time (int): シミュレーションの最大時刻（0～max_time、サイズは max_time+1）
        results_dir (str): CSV ファイルが存在するディレクトリ（デフォルトはカレントディレクトリ）
        
    Returns:
        np.ndarray: 形状 (num_alpha, num_lambda, num_batches*iterations, max_time+1) の連結データ
    """
    total_iterations = num_batches * iterations
    concatenated_data = np.empty((num_alpha, num_lambda, total_iterations, max_time+1), dtype=int)
    
    current_iter_index = 0
    for batch in range(1, num_batches + 1):
        batch_data = load_simulation_result(file_pattern, batch, num_alpha, num_lambda, iterations, max_time, results_dir)
        concatenated_data[:, :, current_iter_index:current_iter_index+iterations, :] = batch_data
        current_iter_index += iterations
    return concatenated_data

if __name__ == "__main__":
    # シミュレーション引数を args.csv から読み込む
    args_csv_path = "args.csv"
    alpha_values, lambda_values, iterations, max_time = load_simulation_args(args_csv_path)
    print("Alpha values:", alpha_values)
    print("Lambda values:", lambda_values)
    print("Iterations per batch:", iterations)
    print("Max time:", max_time)
    
    # パラメータ設定
    num_batches = 20
    num_alpha = len(alpha_values)
    num_lambda = len(lambda_values)
    
    # 結果ファイルが保存されているディレクトリ（Java 側と合わせる）
    results_directory = "/Users/username/Downloads/dir"
    
    # s_all の結果を全バッチ分読み込み、連結する
    s_all = load_all_batches("s_all_results.csv", num_batches, num_alpha, num_lambda, iterations, max_time, results_directory)
    print("s_all shape:", s_all.shape)
